In [117]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss

import glob
import csv
import cv2
from numpy import array, asarray, ndarray, swapaxes
from sklearn.preprocessing import MultiLabelBinarizer


In [118]:
#!pip install torchvision

In [119]:
#!pip install opencv-python

In [120]:
#training controls
batch_size = 20
epochs = 2
training_size = 0.7
learning_rate = 0.001
dropout = [0.3, 0.3, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.15]

# input image dimensions
img_rows, img_cols = 268, 182

In [121]:
# the data holders
x_test = []
x_train = []
y_test= []
y_train= []
tempY = []

In [122]:
#opening the dataset
dataset = csv.reader(open("MovieGenre.csv",encoding="utf8",errors='replace'), delimiter=",")

#skipping the header line
next(dataset)

['\ufeffimdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster']

In [123]:
import zipfile as zf
files = zf.ZipFile("SampleMoviePosters.zip", 'r')
files.extractall()
files.close()

In [124]:
#the list of image files in SampleMoviePosters folder
flist=glob.glob('SampleMoviePosters/*.jpg')  

In [125]:
len(flist)

997

In [126]:
image_ids = []

for path in flist:
    start = path.rfind("\\")+1
    end = len(path)-4
    image_ids.append(path[start:end])

In [127]:
image_ids

['10040',
 '10057',
 '10071',
 '10155',
 '10195',
 '10208',
 '10247',
 '10267',
 '10281',
 '10307',
 '10323',
 '10418',
 '10726',
 '10747',
 '10806',
 '10821',
 '11000',
 '11071',
 '11130',
 '11157',
 '11193',
 '11237',
 '11293',
 '11358',
 '11362',
 '11370',
 '11387',
 '11439',
 '11508',
 '11521',
 '11541',
 '11565',
 '11652',
 '11656',
 '11723',
 '11833',
 '11841',
 '11865',
 '11870',
 '11904',
 '11908',
 '11909',
 '11960',
 '11979',
 '11984',
 '12027',
 '12136',
 '12190',
 '12224',
 '12253',
 '12255',
 '12278',
 '12281',
 '12304',
 '12349',
 '12364',
 '12408',
 '12465',
 '12486',
 '12494',
 '12499',
 '12512',
 '12532',
 '12543',
 '12570',
 '12645',
 '12651',
 '12675',
 '12752',
 '12763',
 '12881',
 '12938',
 '12945',
 '13025',
 '13055',
 '13059',
 '13075',
 '13086',
 '13099',
 '13140',
 '13158',
 '13201',
 '13257',
 '13367',
 '13372',
 '13386',
 '13422',
 '13427',
 '13442',
 '13472',
 '13496',
 '13501',
 '13556',
 '13571',
 '13579',
 '13597',
 '13626',
 '13662',
 '13674',
 '13688',


In [128]:
import pandas as pd

dataset2 = pd.read_csv("MovieGenre.csv")
dataset2

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...
40103,83168,http://www.imdb.com/title/tt83168,Tanya's Island (1980),4.3,Drama,https://images-na.ssl-images-amazon.com/images...
40104,82875,http://www.imdb.com/title/tt82875,Pacific Banana (1981),4.7,Comedy,https://images-na.ssl-images-amazon.com/images...
40105,815258,http://www.imdb.com/title/tt815258,Werewolf in a Womens Prison (2006),4.5,Horror,https://images-na.ssl-images-amazon.com/images...
40106,79142,http://www.imdb.com/title/tt79142,Xiao zi ming da (1979),6.5,Action|Comedy,https://images-na.ssl-images-amazon.com/images...


In [129]:
y = []
indexlist = []
classes = tuple()
ids = dataset2.imdbId.values.tolist()
for image_id in image_ids:
    genres = tuple((dataset2[dataset2["imdbId"] == int(image_id)]["Genre"].values[0]).split("|"))
    if int(image_id) in ids:
        indexlist.append(image_id)
    y.append(genres)
    classes = classes + genres
mlb = MultiLabelBinarizer()
mlb.fit(y)
y = mlb.transform(y)
classes = set(classes)
classes = list(classes)
classes.sort()

In [130]:
y_df = pd.DataFrame(y, columns = classes, index = indexlist)
y_df

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,Music,Musical,Mystery,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
10040,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10057,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10071,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10155,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10195,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9899,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9932,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
9937,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
9968,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [131]:
y_df_reset = y_df.reset_index()

shape = y_df_reset.shape[1]

index_value = []
genre_lst = []

for i in range(len(y_df_reset)):
    index_value.append(int(y_df_reset.loc[i,"index"]))
    temp_list = []
    for j in y_df_reset.columns[1:]:
        temp_list.append(y_df_reset.loc[i,j])
    genre_lst.append(temp_list)

df = pd.DataFrame(list(zip(index_value, genre_lst)),
               columns =['imdbId', 'genrelst'])

result = dataset2.merge(df, on="imdbId")
result

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster,genrelst
0,24252,http://www.imdb.com/title/tt24252,Liebelei (1933),7.7,Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,25316,http://www.imdb.com/title/tt25316,It Happened One Night (1934),8.2,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,25164,http://www.imdb.com/title/tt25164,The Gay Divorcee (1934),7.6,Comedy|Musical|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,17350,http://www.imdb.com/title/tt17350,The Scarlet Letter (1926),7.8,Drama,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,25586,http://www.imdb.com/title/tt25586,Of Human Bondage (1934),7.3,Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
1013,23797,http://www.imdb.com/title/tt23797,Betty Boop's Big Boss (1933),6.6,Animation|Short|Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1014,22670,http://www.imdb.com/title/tt22670,Betty Boop's Museum (1932),6.9,Animation|Short|Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1015,19640,http://www.imdb.com/title/tt19640,Angora Love (1929),7.0,Comedy|Short,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1016,24676,http://www.imdb.com/title/tt24676,TÌ«kyÌ« no onna (1933),7.2,Drama,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [132]:
#extracting the data from the CSV file
#for imdbId, Link, Title, Score, Genre, Poster in dataset2:
#    if(Score!=""):
#        if(len((int(imdbId),float(Score)))==2):
#            tempY.append((int(imdbId),float(Score))) # from Score to array of response binarys
for x in range(len(result)):
    tempY.append((int(result['imdbId'].iloc[x]),result['genrelst'].iloc[x]))

tempY

[(24252,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 (25316,
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 (25164,
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]),
 (17350,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 (25586,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 (22879,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]),
 (23969,
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]),
 (22100,
  [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 (13442,
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 (18578,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]),
 (19729,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]),
 (20629,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
 (21

In [133]:
#setting the length of training data
length=int(len(flist)*training_size)
length

697

In [134]:
#extracting the data about the images that are available
i=0
for filename in flist:
    name=int(filename.split('\\')[-1][:-4])
    for z in tempY:
        if(z[0]==name):
            
            img = array(cv2.imread(filename))
            img = swapaxes(img, 2,0)
            img = swapaxes(img, 2,1)

            if(i<length):
                x_train.append(img)
                y_train.append(z[1])
                i+=1
            else:
                x_test.append(img)
                y_test.append(z[1])
                i+=1

In [135]:
#converting the data from lists to numpy arrays
x_train=asarray(x_train,dtype=float)
x_test=asarray(x_test,dtype=float)
y_train=asarray(y_train,dtype=float)
y_test=asarray(y_test,dtype=float)

In [136]:
#scaling down the RGB data
x_train /= 255
x_test /= 255

In [137]:
#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (697, 3, 268, 182)
697 train samples
321 test samples


In [138]:
train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

In [139]:
class Net(nn.Module):
    def __init__(self, input_shape=(3, img_rows, img_cols)):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, kernel_size=3)
        self.conv1_drop = nn.Dropout2d(p=dropout[0])
        self.conv2 = nn.Conv2d(128, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d(p=dropout[1])
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3)
        self.conv3_drop = nn.Dropout2d(p=dropout[2])
        self.conv4 = nn.Conv2d(64, 64, kernel_size=2)
        self.conv4_drop = nn.Dropout2d(p=dropout[3])
        self.conv5 = nn.Conv2d(64, 32, kernel_size=2)
        self.conv5_drop = nn.Dropout2d(p=dropout[4])
        self.conv6 = nn.Conv2d(32, 16, kernel_size=2)
        self.conv6_drop = nn.Dropout2d(p=dropout[5])
        
        n_size = self._get_conv_output(input_shape)
        
        self.fc1 = nn.Linear(n_size, 16)
        self.fc1_drop = nn.Dropout(p=dropout[6])
        self.fc2 = nn.Linear(16, 16)
        self.fc2_drop = nn.Dropout(p=dropout[7])
        self.fc3 = nn.Linear(16, 8)
        self.fc3_drop = nn.Dropout(p=dropout[8])
        self.fc4 = nn.Linear(8, 1)
        
    def _get_conv_output(self, shape):
        bs = 1
        input = Variable(torch.rand(bs, *shape))
        output_feat = self._forward_features(input)
        n_size = output_feat.data.view(bs, -1).size(1)
        return n_size
        
    def _forward_features(self, x):
        x = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3(x)), 2))
        x = F.relu(F.max_pool2d(self.conv4_drop(self.conv4(x)), 2))
        x = F.relu(F.max_pool2d(self.conv5_drop(self.conv5(x)), 2))
        x = F.relu(F.max_pool2d(self.conv6_drop(self.conv6(x)), 2))
        return x
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3(x)), 2))
        x = F.relu(F.max_pool2d(self.conv4_drop(self.conv4(x)), 2))
        x = F.relu(F.max_pool2d(self.conv5_drop(self.conv5(x)), 2))
        x = F.relu(F.max_pool2d(self.conv6_drop(self.conv6(x)), 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1_drop(self.fc1(x)))
        x = F.relu(self.fc2_drop(self.fc2(x)))
        x = F.relu(self.fc3_drop(self.fc3(x)))
        x = self.fc4(x)
        return x


In [140]:
model = Net()
criterion = MSELoss(size_average=True) #change to BCE loss
human_criterion = L1Loss(size_average=True)
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate,
            alpha=0.9, eps=1e-08, weight_decay=0.0)

C:\Users\royal\anaconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        human_loss= human_criterion(output, target)
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), human_loss.data.item()))

def test():
    print('test')
    model.eval()
    test_loss = 0
    correct = 0
    human_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        data, target = Variable(data, volatile=True).float(), Variable(target).float()
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss
        human_loss += human_criterion(output, target)
        if loss==0:
            correct+=1

    print('\nTest set: \nAverage sq_loss: {:.4f} \nAverage abs_loss: {:.4f} \nGuessed 100% correct: {:.4f}\n'.format(test_loss.data.item()/i, human_loss.data.item()/i, correct))

model.float()
print(model)
for epoch in range(0, epochs):
    train(epoch)
    test()

Net(
  (conv1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv1_drop): Dropout2d(p=0.3, inplace=False)
  (conv2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.3, inplace=False)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3_drop): Dropout2d(p=0.3, inplace=False)
  (conv4): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1))
  (conv4_drop): Dropout2d(p=0.3, inplace=False)
  (conv5): Conv2d(64, 32, kernel_size=(2, 2), stride=(1, 1))
  (conv5_drop): Dropout2d(p=0.2, inplace=False)
  (conv6): Conv2d(32, 16, kernel_size=(2, 2), stride=(1, 1))
  (conv6_drop): Dropout2d(p=0.2, inplace=False)
  (fc1): Linear(in_features=48, out_features=16, bias=True)
  (fc1_drop): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc2_drop): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc3_drop): Dropout(p=0.15, inplace=False)
  (fc4): Linear(in_features=

C:\Users\royal\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([20, 23])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\royal\anaconda3\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([20, 23])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Train Epoch: 0 [0/697 (0%)]	Loss: 0.099625 0.253906
Train Epoch: 0 [20/697 (3%)]	Loss: 0.094254 0.208634
Train Epoch: 0 [40/697 (6%)]	Loss: 0.072878 0.146103
Train Epoch: 0 [60/697 (9%)]	Loss: 0.094987 0.174298
Train Epoch: 0 [80/697 (11%)]	Loss: 0.094210 0.178797
Train Epoch: 0 [100/697 (14%)]	Loss: 0.086960 0.176967
Train Epoch: 0 [120/697 (17%)]	Loss: 0.076923 0.162943
Train Epoch: 0 [140/697 (20%)]	Loss: 0.083197 0.160458
Train Epoch: 0 [160/697 (23%)]	Loss: 0.086762 0.165714
Train Epoch: 0 [180/697 (26%)]	Loss: 0.081682 0.168999
Train Epoch: 0 [200/697 (29%)]	Loss: 0.079491 0.162442
